In [1]:
from dataclasses import dataclass
import shelve
from datetime import datetime
import random
from scripts import utils
from pprint import pprint
import hashlib
import os
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
plt.style.use('dark_background')

In [2]:
DF_PATH = './benchmarking-dataset/runtime-data.parquet'

## Data Visualization

In [3]:
df = pl.read_parquet(DF_PATH) \
    # .filter(pl.col('c_filename') == 'n-bodies-753227e0.c')
	# .filter(pl.col('flags').is_in(["-O3", "-O3 -fwhole-program"])) \
df.sort(pl.col('runtime'))

c_filename,flags,runtime,created_at,trials,sys_specs_hash,sys_specs_os,sys_specs_cpu,sys_specs_compiler,sys_specs_os_version,sys_specs_cpu_arch,sys_specs_cpu_physical_cores,sys_specs_cpu_logical_cores,sys_specs_mem_size_gb,sys_specs_mem_speed_mhz,sys_specs_l1_i_size,sys_specs_l1_d_size,sys_specs_l2_size,sys_specs_l3_size,id
str,str,f64,datetime[μs],i64,str,str,str,str,str,str,i64,i64,f64,null,str,str,str,str,str
"""eigenvalue-33c06ce6.c""","""-O3 -march=native -mtune=nativ…",0.304516,2024-11-03 22:30:10.171520,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""948bddda92e7f5e374c0cfa1049165…"
"""eigenvalue-33c06ce6.c""","""-O3 -funroll-loops -fprefetch-…",0.304858,2024-11-03 22:29:59.796546,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""fa886d31e2da16d92dea8e4bed3c39…"
"""eigenvalue-33c06ce6.c""","""-O3 -funroll-loops -march=nati…",0.41399,2024-11-03 22:29:59.449327,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""9bef08bfb4fff715a387f79d80c043…"
"""eigenvalue-33c06ce6.c""","""-O3 -march=native -funroll-loo…",0.414015,2024-11-03 22:29:50.755481,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""52a4971819d05b87a67712f029741b…"
"""eigenvalue-33c06ce6.c""","""-O3 -ffast-math -funroll-loops…",0.41501,2024-11-03 22:29:57.171162,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""2fd2e51049c878750c7cb46ef0de9c…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""cond-branching-87209a89.c""","""-O3 -flto -fwhole-program -mar…",4.4576,2024-11-03 21:43:32.089278,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""c4b3cb18925e849d4cc904262ff496…"
"""cond-branching-87209a89.c""","""-O3 -mtune=native -funroll-loo…",4.467,2024-11-03 21:42:57.122584,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""947b96015fe6d7ad79eb5ad66391e7…"
"""cond-branching-87209a89.c""","""-O3 -funroll-loops -flto""",4.469425,2024-11-03 21:41:33.502398,4,"""04c116405dd7c5743463fa90b64467…","""Linux""","""AMD Ryzen 9 3950X 16-Core Proc…","""gcc (Ubuntu 11.4.0-1ubuntu1~22…","""Linux-6.9.3-76060903-generic-x…","""x86_64""",16,32,32.780768,null,"""512 KiB (16 instances)""","""512 KiB (16 instances)""","""8 MiB (16 instances)""","""64 MiB (4 instances)""","""28b31a0705d2555865059d86cb0425…"


In [11]:
df_grouped = df[['flags', 'runtime']].group_by(pl.col('flags')).mean()
df_grouped = df_grouped.sort(pl.col('runtime'))
df_grouped

flags,runtime
str,f64
"""-O3 -march=native -mtune=nativ…",1.67875
"""-O3 -funroll-loops -fprefetch-…",1.684625
"""-O3 -fno-math-errno -funsafe-m…",1.68826
"""-O3 -funroll-loops -march=nati…",1.692614
"""-O3 -march=native -funroll-loo…",1.695994
…,…
"""-O3 -falign-functions=32""",1.885852
"""-O3 -falign-loops=32""",1.886399
"""-O3""",1.88736


In [12]:
df_grouped['flags'].to_list()

['-O3 -march=native -mtune=native -funroll-loops -fprefetch-loop-arrays -flto -ffast-math -fno-math-errno -funsafe-math-optimizations -fomit-frame-pointer -fwhole-program -fmerge-all-constants -fipa-pta -falign-functions=32 -falign-loops=32',
 '-O3 -funroll-loops -fprefetch-loop-arrays -march=native -ffast-math',
 '-O3 -fno-math-errno -funsafe-math-optimizations -march=native -funroll-loops',
 '-O3 -funroll-loops -march=native -mtune=native -fno-math-errno -ffast-math',
 '-O3 -march=native -funroll-loops -funsafe-math-optimizations -ffast-math',
 '-O3 -march=native -funroll-loops -ffast-math',
 '-O3 -fwhole-program -funroll-loops -march=native -ffast-math',
 '-O3 -ffast-math -funroll-loops -fwhole-program -march=native',
 '-O3 -funsafe-math-optimizations -fomit-frame-pointer -funroll-loops -flto',
 '-O3 -ffast-math -funroll-loops -fomit-frame-pointer -flto',
 '-O3 -funroll-loops -funsafe-math-optimizations -flto',
 '-O3 -mtune=native -funroll-loops -funsafe-math-optimizations',
 '-O3 -

In [10]:
df['flags'].unique()

flags
str
"""-O3 -funroll-loops -fprefetch-…"
"""-O3 -fwhole-program"""
"""-O3 -ffast-math -fno-math-errn…"
"""-O3 -ffast-math -fno-math-errn…"
"""-O3 -ftree-vectorize"""
…
"""-O3 -fno-math-errno -funsafe-m…"
"""-O3 -march=native"""
"""-O3 -falign-functions=32"""


## Scratch

In [2]:
utils.SystemSpec.generate()

SystemSpec(os='Linux', cpu='AMD Ryzen 9 3950X 16-Core Processor', compiler='gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0', os_version='Linux-6.9.3-76060903-generic-x86_64-with-glibc2.35', cpu_arch='x86_64', cpu_physical_cores=16, cpu_logical_cores=32, mem_size_gb=32.780768, mem_speed_mhz=None, l1_i_size='512 KiB (16 instances)', l1_d_size='512 KiB (16 instances)', l2_size='8 MiB (16 instances)', l3_size='64 MiB (4 instances)')

In [ ]:
with shelve.open('./benchmarking-dataset/benchmark.shelve') as db:
    data = dict(db)
len(data)

In [ ]:
idx = 9
ptr: utils.ExecutionTrial = list(data.values())[idx]
print(ptr.c_filename)
print(ptr.flags)
print(ptr.time_elapsed())

In [4]:
flags = [
	# Control
	"-O3",

	# Base optimization flags
	"-O3 -funroll-loops",
	"-O3 -fprefetch-loop-arrays",
	"-O3 -flto",
	"-O3 -ffast-math",
	"-O3 -fno-math-errno",
	"-O3 -funsafe-math-optimizations",
	"-O3 -march=native",
	"-O3 -mtune=native",
	"-O3 -fomit-frame-pointer",
	"-O3 -fwhole-program",
	"-O3 -falign-loops=32",
	"-O3 -falign-functions=32",

	# Combinations of optimization flag
	"-O3 -funroll-loops -flto",                              
	"-O3 -march=native -mtune=native -flto",                 
	"-O3 -funroll-loops -fomit-frame-pointer -flto",         
	"-O3 -ffast-math -fno-math-errno -funsafe-math-optimizations",  
	"-O3 -march=native -funroll-loops -ffast-math",          
	"-O3 -march=native -fomit-frame-pointer -flto",          
	"-O3 -fwhole-program -flto -march=native",               
	"-O3 -funroll-loops -funsafe-math-optimizations -flto",  
	"-O3 -funroll-loops -fno-math-errno -march=native",
	"-O3 -fwhole-program -funroll-loops -march=native -ffast-math", 

	# GPT thought these would be good
	"-O3 -march=native -mtune=native -funsafe-math-optimizations", # Native CPU with unsafe math optimizations
	"-O3 -flto -fomit-frame-pointer -fno-math-errno",              # LTO, no frame pointer, math optimizations
	"-O3 -funroll-loops -march=native -fwhole-program -flto",      # Unrolling, whole program, and LTO
	"-O3 -mtune=native -funroll-loops -funsafe-math-optimizations",# CPU-tuned loop unrolling and unsafe math
	"-O3 -funroll-loops -fprefetch-loop-arrays -flto",             # Loop unrolling, prefetching, and LTO
	"-O3 -ffast-math -funroll-loops -fomit-frame-pointer -flto",   # Fast math, unrolling, omit frame, and LTO
	"-O3 -march=native -fwhole-program -funsafe-math-optimizations",# Whole program with unsafe math and native tuning
	"-O3 -march=native -funroll-loops -funsafe-math-optimizations -ffast-math", # Native loop unrolling with aggressive math
	"-O3 -fwhole-program -flto -fomit-frame-pointer -funsafe-math-optimizations",# Whole program, LTO, omit frame, unsafe math
	"-O3 -mtune=native -funroll-loops -fwhole-program -flto",      # CPU-tuned loop unrolling, whole program, and LTO
	"-O3 -fno-math-errno -funsafe-math-optimizations -march=native -funroll-loops", # Math optimizations with native tuning and loop unrolling
	"-O3 -march=native -mtune=native -flto -fwhole-program",       # Native tuning, LTO, and whole program
	"-O3 -funsafe-math-optimizations -fomit-frame-pointer -funroll-loops -flto", # Unsafe math, omit frame, unrolling, and LTO
	"-O3 -ffast-math -funroll-loops -fwhole-program -march=native", # Fast math, unrolling, whole program, native tuning
	"-O3 -fprefetch-loop-arrays -funroll-loops -flto",             # Loop prefetching, unrolling, and LTO
	"-O3 -funroll-loops -march=native -mtune=native -fno-math-errno -ffast-math", # Native tuning, loop unrolling, and math optimizations
	"-O3 -funroll-loops -fprefetch-loop-arrays -march=native -ffast-math", # Prefetch, loop unroll, native tuning, fast math
	"-O3 -flto -fwhole-program -march=native -funroll-loops",      # LTO, whole program, native tuning, loop unrolling
	"-O3 -ffast-math -fno-math-errno -fomit-frame-pointer -flto",  # Fast math, omit frame, and LTO
	"-O3 -march=native -fomit-frame-pointer -fwhole-program -flto",# Native tuning, omit frame, whole program, LTO
	"-O3 -funroll-loops -fprefetch-loop-arrays -march=native -flto", # Loop prefetching, unroll, native tuning, LTO
	"-O3 -march=native -mtune=native -funroll-loops -fprefetch-loop-arrays -flto -ffast-math -fno-math-errno -funsafe-math-optimizations -fomit-frame-pointer -fwhole-program -fmerge-all-constants -fipa-pta -falign-functions=32 -falign-loops=32"

	# Claude thought these could be good
	# Vectorization-focused optimizations
	"-O3 -ftree-vectorize -fopt-info-vec"  # Explicit vectorization with feedback
	"-O3 -ftree-vectorize -fopt-info-vec-missed" # Help identify vectorization opportunities
	# Memory and cache optimizations
	"-O3 -fmodulo-sched -fmodulo-sched-allow-regmoves" # Software pipelining
	"-O3 -fpeel-loops -funswitch-loops"  # Additional loop transformations
	"-O3 -ftracer" # Tail duplication to avoid branching
	# Aggressive optimizations
	"-O3 -fgcse-after-reload" # Additional global CSE after reload
	"-O3 -fipa-pta -fdevirtualize-at-ltrans" # More aggressive devirtualization
	"-O3 -fgraphite-identity -floop-nest-optimize" # Polyhedral optimizations
	# Combined aggressive optimizations
	"-O3 -march=native -ftracer -fmodulo-sched -funswitch-loops -flto"
	"-O3 -march=native -ftree-vectorize -fmodulo-sched -ffast-math -flto"
	"-O3 -fgraphite-identity -floop-nest-optimize -march=native -ffast-math -flto"
]
len(flags) * 17 * 4


3060